In [1]:
from transformers import pipeline
import pandas as pd
import tensorflow as tf

# Preparing data
path = '../data/songs/'

music_df = pd.concat([pd.read_csv(path+'youtube_extract_MrSuicideSheep.csv', sep='\t'), pd.read_csv(path+'youtube_extract_tameimpalaVEVO.csv', sep='\t')])
non_music_df = pd.concat([pd.read_csv(path+'youtube_extract_Mia_Maria.csv', sep='\t'), pd.read_csv(path+'youtube_extract_Payo.csv', sep='\t'), pd.read_csv(path+'youtube_extract_The_Swedish_Investor.csv', sep='\t'),
                         pd.read_csv(path+'youtube_extract_videogamedunkey.csv', sep='\t')])
# adding targets, 1 is song else 0
music_df['is_song'] = 1
non_music_df['is_song'] = 0
df = music_df[['title', 'description', 'is_song']].append(non_music_df[['title', 'description', 'is_song']], ignore_index = True)

In [2]:
# Doing text cleaning
import spacy
import re

nlp = spacy.load("en_core_web_md", disable=['parser', 'tagger', 'ner'])
# adding some custom stop words
nlp.vocab["mrsuicidesheep"].is_stop = True

def my_better_tokenizer(text):

    doc = nlp(text)

    # Create a list of tokens using nlp and the sample string "text"
    tokens = []

    # iterate through the tokens in the doc
    for token in doc:

        # create a couple of filters for low quality tokens
        if (token.is_stop != True) and (token.is_punct != True):
            # save case normalized lemmas to token list
            tokens.append(token.lemma_.lower())

    return tokens

In [3]:
df['tokenized'] = df['description'].apply(lambda x: my_better_tokenizer(x))
df['tokenized']

TypeError: object of type 'float' has no len()

In [ ]:
# Testing a vanilla zero-shot song classification algorithm 

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
df_pred = df.copy()
candidate_labels = ['music', 'other']
for i in df.shape[0]:
    dict = classifier(df['title'][i], candidate_labels)
    df_pred['pred'] = dict['scores'][0]

['sequence', 'labels', 'scores']


In [11]:
# training a NER to try distinguish artists and song titles

ner_classifier = pipeline("ner", grouped_entities=True)
ner_test = {}
for i in range(10):
    ner_test[ten_video_names[i]] = ner_classifier(ten_video_names[i])

Some layers from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing TFBertForTokenClassification: ['dropout_147']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.
/opt/anaconda3/envs/utube/l

In [15]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [12]:
ner_test


{'Hanne Mjøen - Hell With You (lyrics)': [{'entity_group': 'PER',
   'score': 0.96508867,
   'word': 'Hanne Mjøen',
   'start': 0,
   'end': 11},
  {'entity_group': 'MISC',
   'score': 0.82035536,
   'word': 'Hell With You',
   'start': 14,
   'end': 27}],
 'Arrested Youth - Leave My Casket Open... (SMLE Remix)': [{'entity_group': 'ORG',
   'score': 0.9781659,
   'word': 'Arrested Youth',
   'start': 0,
   'end': 14},
  {'entity_group': 'MISC',
   'score': 0.70439076,
   'word': 'Leave My Casket Open',
   'start': 17,
   'end': 37},
  {'entity_group': 'ORG',
   'score': 0.89473087,
   'word': 'SMLE',
   'start': 42,
   'end': 46}],
 'Aether - Fragments': [],
 'Justin Bieber - Peaches (Marc Benjamin Remix)': [{'entity_group': 'PER',
   'score': 0.9424185,
   'word': 'Justin Bieber',
   'start': 0,
   'end': 13},
  {'entity_group': 'MISC',
   'score': 0.94609016,
   'word': 'Peaches',
   'start': 16,
   'end': 23},
  {'entity_group': 'PER',
   'score': 0.98573947,
   'word': 'Marc Benjam